# Non-Negative Matrix Factorization

Estimated time needed: **45** minutes





You've been hired by a video streaming platform that specializes in kids content. The company would like to ensure that all content on the platform does not infringe on a copyright written material. The company has created an image database of copyrighted material, not only must you detect the same images you must detect similar images. Your  job is to come up with an image retrieval system to find the most similar image to each image in the provided dataset. Because your model is used for legal purposes it must be interpretable, this means it should be able to determine why the images are similar. As a result, you will use Non-Negative Matrix factorization since the factorized matrices can be interpreted as real images. You will use the [Anime Face Dataset](https://www.kaggle.com/datasets/splcher/animefacedataset?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01) from kaggle.



<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/fake_cartoon.png" alt="cognitiveclass.ai logo">

<a href="https://www.piratesandprincesses.net/the-story-of-goodtimes-and-their-numerous-disney-knockoffs/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01"> image source</a>


## Table of Contents

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li><a href="#Datasets">Datasets</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li>
        <a href="#Background">Background</a>
        <ol>
               <li><a href="#What's a Non-Negative Matrix Fatorization?">What's a Non-Negative Matrix Fatorization?</a></li>
        </ol>
    </li>
  <li><a href="#Applying Non-Negative Matrix Factorization">Applying Non-Negative Matrix Factorization</a></li>
  <li><a href="#Image Retrieval System">Image Retrieval System</a></li>
   <li><a href="#Exercises">Exercises</a>
       <ol>
            <li><a href="#Exercise 1">Exercise 1</a></li>
            <li><a href="#Exercise 2">Exercise 2</a></li>
            <li><a href="#Exercise 3">Exercise 3</a></li>
            <li><a href="#Exercise 4">Exercise 4</a></li>
            <li><a href="#Exercise 5">Exercise 5</a></li>
        </ol>
    </li> 
 </ol>    
     
    
     

## Objectives

After completing this lab you will be able to:

- __Understand__ Basics Matrix operations  such as Matrix addition  , Vector Multiplication，and Eigen decomposition

- __Apply__ Apply these Matrix operations using numpy


***


## Datasets

Download and unzip the **images** dataset: 


In [ ]:
import skillsnetwork

await skillsnetwork.prepare("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/images.tar.gz")

## Setup


For this lab, we will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01) for mathematical operations.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01) for visualizing the data.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01) for visualizing the data.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork34171862-2022-01-01) for machine learning and machine-learning-pipeline related functions.


### Installing required libraries


The following required modules are pre-installed in the Skills Network Labs environment. However, if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you will need to install these libraries by removing the `#` sign before `!mamba` in the code cell below.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# Note: If your environment doesn't support "!mamba install", use "!pip install"


you can use  <a href="https://www.sympy.org/en/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01">SymPy </a>this to print matrices 


In [ ]:
#!conda install -c anaconda sympy -y

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [ ]:
import warnings
warnings.simplefilter('ignore')

import logging
import numpy as np
import pandas as pd
from numpy.random import RandomState
import matplotlib.pyplot as plt


from sklearn.decomposition import NMF


from os import listdir,getcwd
from os.path import isfile, join
from PIL import Image, ImageOps
import os 


### Defining Helper Functions


Load train and test set:


In [ ]:
def get_data_matrix(test=False,Length=100,Width=100,mypath="images/"):

    files = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f)) and f[0] != '.']
    if mypath + '/.DS_Store' in files:
        files.remove(mypath + '/.DS_Store')
  
    if test:
        print("test data")
        files=files[9000:10000]
        
    else:
        print("training data")
        files=files[0:9000]
        
    print(len(files))
    X=np.zeros((len(files),Length*Width))
    for i,file in enumerate(files):
        img = Image.open(file).resize((Width, Length))
        img =  ImageOps.grayscale(img)

        I=np.array(img)
 
        X[i,:]=I.reshape(1,-1)
    return X

Plot image vectors:


In [ ]:
def reshape_row(x) :
    plt.imshow(x.reshape(Length,Width),cmap="gray")
    

This threshold function outputs an index if the input ```similar_distance``` is grater than or less than ```min_``` and ```max_```.


In [ ]:
def threshold(similar_distance,max_=0.1,min_=0):
    dataset_index=np.where(np.logical_and(similar_distance>min_ ,similar_distance<max_))[0]
    query_index=similar_index[np.logical_and(similar_distance>min_ ,similar_distance<max_)]
    return dataset_index,query_index

Plot dataset images and query images, ```X``` and ```X_q``` are global variables.


In [ ]:
def plot_data_query(dataset_index,query_index,N):
    for data_sample,query_sample in zip(dataset_index[0:N],query_index[0:N]):

        plt.figure(figsize=(10,4))
        plt.subplot(1,2,1)
        reshape_row(X[data_sample])
        plt.title("dataset sample {}".format(data_sample))
        plt.subplot(1,2,2)
        reshape_row(X_q[query_sample])
        plt.title("query sample match {}".format(query_sample))
        plt.show()

        print("-----------------------------------------------------")

## Background


### What's a Non-Negative Matrix Factorization?


In Non-Negative Matrix Factorization (NNMF) sometimes called non-negative matrix approximation, a $N x D$ matrix $\mathbf{X}$, where $N$ is the number of samples and $D$ is the dimension of features, contains all non-negative values. In NNMF we construct a non-negative  approximation of $\mathbf{X}$, $\mathbf{\hat{X}}$ such that $\mathbf{\hat{X}}$ is a product of two non-negative matrices $\mathbf{W}$ and $\mathbf{H}$, i.e:

$$\mathbf{\hat{X}}=\mathbf{WH}$$

where the $r$ rows of $\mathbf{H}$ are called the basis, and we are going to represent the matrix $\mathbf{X}$ in terms of this basis. 

Each row of $\mathbf{W}$, ${w}_i$ is called an encoding of the corresponding ${x}_i$ in $\mathbf{X}$, which represents "how much" of each basis in $\mathbf{H}$ is to represent ${x}_i$. ${\hat{x}_i}$ is thus a linear combination of the rows of $\mathbf{H}$.


## Applying Non-Negative Matrix Factorization


**Image retrieval** is used for searching and retrieving images from a large database of digital images. Here we would like to find identical or similar  animated faces  for copyright reasons.


Consider the matrix $\mathbf{X}$ where each row is a flattened $100\times 100$ image.


In [ ]:
Length,Width=100,100
X=get_data_matrix(test=False,Length=100,Width=100,mypath="images")
X.shape

We can apply ```sklearn.decomposition.NMF``` to obtain the  non-negative matrices, i.e. matrices with all non-negative elements, $(\mathbf{W}$, $\mathbf{H})$ whose product approximates the non-negative matrix $\mathbf{X}$. 

Consider the following parameters when building a simple NMF object: 

```n_components```: int; Number of components, if n_components is not set all features are kept.

```tolfloat```: default=1e-4; Tolerance of the stopping condition.

```max_iter```: int, (default=200); Maximum number of iterations before timing out.


Let's select a number of components or basis, namely $r$, for the matrix representation.


In [ ]:
n_components=10

Initialize our NMF object and fit it to $\mathbf{X}$:


In [ ]:
nmf_estimator = NMF(n_components=n_components, tol=5e-3,max_iter=2000)
nmf_estimator.fit(X)  # original non- negative dataset

The **.components_** method returns the basis found:


In [ ]:
H = nmf_estimator.components_

We can reshape the obtained 10 basis into images, and they appear like faces in the dataset:


In [ ]:
plt.figure(figsize=(25, 8))

for i,h in enumerate(H):   
    plt.subplot(2, 5, i+1)
    reshape_row(h)
    plt.title("basis images {}".format(str(i+1)), fontsize=20) 
    
plt.tight_layout()

We can obtain the encodings of all the images using the **transform** method of the fitted **nmf_estimator**:


In [ ]:
W = nmf_estimator.transform(X)

For each of the 9000 images we have an encoding of dimension 10:


In [ ]:
W.shape

The encoding tells you the projection of each image in $\mathbf{X}$ on a particular basis. Here we plot out the encoding magnitude for the $i$th image on each basis.


In [ ]:
i=0
w=W[i,:]

plt.bar([n+1 for n in range(len(w))],w)
plt.title("encodings for image {} ".format (i+1))

We see that for image 1, the second encoding has the highest magnitude, thus we can plot out the basis associated with this encoding (i.e, the second basis) and see that it's very similar to the image. The 9th encoding has a small magnitude, we see the associated basis looks nothing like the image:


In [ ]:
plt.figure(figsize=(15,4))

plt.subplot(131)
reshape_row(X[i,:])
plt.title("Original image")

plt.subplot(132)
reshape_row(H[1,:])
plt.title("Similar basis 2")

plt.subplot(133)
reshape_row(H[8,:])
plt.title("Dissimilar basis 9")

We can perform the inverse transform using the method **inverse_transform** to get $\mathbf{\hat{X}}$, the approximation of $\mathbf{X}$:


In [ ]:
Xhat=nmf_estimator.inverse_transform(W)

Each row of $\mathbf{\hat{X}}$ corresponds to an image. We can reshape the row into an image and plot the approximations:


In [ ]:
plt.figure(figsize=(20,8))

for i in range(1,5):
    plt.subplot(2,4,i)
    reshape_row(X[i])
    plt.title(f"Original image {i}")
    
    plt.subplot(2,4,i+4)
    reshape_row(Xhat[i])
    plt.title(f"Approximated image {i}")

We can verify that the scikit-learn's **inverse_transform** method essentially performs the following matrix operation:


In [ ]:
Xhat_M=W@H

We see the results are identical quantitatively (for the first 10 grayscale values of first image):


In [ ]:
Xhat[0,:10], Xhat_M[0,:10]

and qualitatively or visually for the second to fifth image in the dataset: 


In [ ]:
plt.figure(figsize=(20,8))

for i in range(1,5):
    plt.subplot(2,4,i)
    reshape_row(Xhat[i])
    plt.title(f"Approximated image {i} with sklearn")
    
    plt.subplot(2,4,i+4)
    reshape_row(Xhat_M[i])
    plt.title(f"Approximated image {i} with matrix operation")

We can reconstruct the image by adding one component multiplied by it's encoding at a time. This is shown in the following lines of code, as each successive component is added we see the image looks more and more like it's approximation:


In [ ]:
# initialize an image array with 10000 zeros which will be reshaped as 100x100  
image=np.zeros((1,10000))

plt.figure(figsize=(25,8))
for i, (w_, h) in enumerate(zip(w, H)):
    
    # w is the encoding vector of the first image in X
    # reconstruction of the image is a linear combination of H 
    plt.subplot(2,5,i+1)
    image += w_*h
    reshape_row(image)
    plt.title(f"{i+1} components added", fontsize=20)
plt.tight_layout()

## Image Retrieval System 


An image retrieval system is a system used for browsing, searching and retrieving images from a large database of digital images. In this section we will create an image retrieval system to find similar cartoon characters for copyright infringement. 

We define the query dataset as the set of 1000 images. If these images are similar to the images in our original dataset, they may be considered as copyright infringement. We load the query dateset into```X_q``` where each row corresponds to a flattened $100\times 100$ image:


In [ ]:
X_q=get_data_matrix(test=True,Length=100,Width=100,mypath="images")
X_q.shape

The code in the next cell helps us retain much of the important information of the query images and reduces factors that make similar images appear differently such as noise, rotation and scale. In addition, the code results in a matrix `W_q` that requires less computation for calculating the distance metrics. As such we will use the code to find the similar images.


In [ ]:
W_q=nmf_estimator.transform(X_q)
W_q.shape

We define  the  distance matrix as a  matrix ```D``` containing the distances, taken pairwise, between the elements of the query dataset and the original dataset.  We will use the cosine similarity that computes the normalized dot product of vectors. That is, for $x$ and $x'$ their cosine similarity  is defined as:

$$1-\frac{x' x^T}{\|x'\| \|x\|}$$


Consider the following set of codes in which case that we represent as 2D vectors.  The original  dataset is in yellow and the query dataset is in blue. We see that the image $h_2$ is nearest  to  $h_{2q}$, and similarly image $h_3$ is nearest  to  $h_{1q}$.



<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/NNMF_vectors.png" width="500" alt="https://pxhere.com/en/photo/1536663">


The resultant distance Matrix is shown below, each row consists of a dataset sample and each column consists of a query dataset sample. Each element of the distance Matrix comprises of the distance between the sample of that particular row and that particular column. We see in this case samples that are close together have the smallest  value. For example, image $h_2$  in the second row is nearest to $h_{2q}$ in the second column.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML0187EN-SkillsNetwork/labs/module%203/images/Sim_Matix.png" width="500" alt="https://pxhere.com/en/photo/1536663">


We will use the function ```pairwise_distances``` from scikit-learn. This method takes either a vector array or a distance matrix, and returns a distance matrix. If the input is a vector array, the distances are computed.


In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
# calculates the pairwise distance between the NNMF encoded version of 
# the original dataset W and the encoded version of the query dataset W_q
D=pairwise_distances(W,W_q,metric='cosine')

We see the number of rows corresponds to the number of samples in the original dataset and the number of columns  corresponds to the number of samples in the query dateset.


In [ ]:
D.shape

For each sample in the dataset we find the query sample that has the smallest distance to it, which would be the most similar image. As each image in the original dataset corresponds to a row in the distance matrix, the column index with smallest value corresponds to the index of the closest image in the query dataset.


In [ ]:
similar_index=np.argmin(D, axis=1)

We also find the corresponding distance value 


In [ ]:
similar_distance=np.min(D, axis=1)

We plot out the distance values in a histogram; we see the values range from zero to approximately 0.06.


In [ ]:
plt.hist(similar_distance,bins=100)
plt.title("Distance values")

If we try a maximum threshold of $1.0x10^{-5}$ for the distance value for the first five samples, we get identical images


In [ ]:
dataset_index,query_index=threshold(similar_distance,max_=0.00001,min_=0)

In [ ]:
dataset_index.shape, query_index.shape

In [ ]:
dataset_index[:10]

In [ ]:
query_index[:10]

If we try a threshold of between $5.0x10^{-3}$ and $1.0x10^{-2}$ instead and plot out the first five samples, it is evident the many of the query images are very different from the original. 


In [ ]:
dataset_index,query_index=threshold(similar_distance,max_=0.005,min_=0.00001)
plot_data_query(dataset_index,query_index,5)

## Exercises


In this exercise, you will apply NMF on the [face images](https://scikit-learn.org/stable/auto_examples/decomposition/plot_faces_decomposition.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork821-2023-01-01#sphx-glr-auto-examples-decomposition-plot-faces-decomposition-py) from scikit-learn and perform an image retrievel task to find pairs of faces that are similar.


In [ ]:
from sklearn.datasets import fetch_olivetti_faces

rng = RandomState(0)
data = fetch_olivetti_faces(shuffle=True, random_state=rng)
X = data.images

In [ ]:
image_shape = (64, 64)

def plot_faces(title, images, n_col, n_row, cmap=plt.cm.gray):
    fig, axs = plt.subplots(
        nrows = n_row,
        ncols = n_col,
        figsize = (2.0*n_col, 2.3*n_row),
        facecolor='white',
        constrained_layout=True)
    
    fig.set_constrained_layout_pads(w_pad=0.01, h_pad=0.02, hspace=0, wspace=0)
    fig.set_edgecolor("black")
    fig.suptitle(title, size=16)
    for ax, vec in zip(axs.flat, images):
        vmax = max(vec.max(), -vec.min())
        im = ax.imshow(
            vec.reshape(image_shape),
            cmap=cmap,
            interpolation="nearest",
            vmin=-vmax,
            vmax=vmax)
        ax.axis('off')
    
    fig.colorbar(im, ax=axs, orientation='horizontal', shrink=0.99, 
                aspect=40, pad=0.01)
    plt.show()


We use the **plot_faces** function to plot the first 6 faces in the dataset.


In [ ]:
plot_faces("6 Faces from the Original Dataset", X[:6], 3, 2)

### Exercise 1

Split the dataset into two such that the first 300 is used as the original dataset **X_r** and the remaining 100 is used as the query dataset **X_q**. Initialize a NMF instance with n_components = 10, and use it to transform **X_r** and **X_q** into encodings.


In [ ]:
X_r = X[:300].reshape((300, 64*64))
X_q = X[300:].reshape((100, 64*64))

nmf = NMF(n_components=10, tol=5e-3,max_iter=2000)
X_r_W = nmf.fit_transform(X_r)
X_q_W = nmf.fit_transform(X_q)

<details>
    <summary>Click here for Solution</summary>

```python
nmf = NMF(n_components=6, tol=5e-3,max_iter=2000)
nmf.fit(X)
```

</details>


### Exercise 2

Calculate the pairwise distances between face encodings from the original and query dataset, store the result in **D**.


In [ ]:
from sklearn.metrics import pairwise_distances

D = pairwise_distances(X_r_W, X_q_W, metric='cosine')

<details>
    <summary>Click here for Solution</summary>

```python
H = nmf.components_
plot_faces("Basis from dataset", H, 3, 2)
```

</details>


### Exercise 3

For all rows in the pairwise distance matrix, output their smallest distances as **similar_distance**, and the index associated with the smallest distances as **similar_index**.


In [ ]:
similar_index = np.argmin(D, axis=1)
similar_distance = np.min(D, axis=1)

<details>
    <summary>Click here for Solution</summary>

```python
W = nmf.transform(X)
w6 = W[:6, :] # picking the first 6 encodings

X_hat = nmf.inverse_transform(w6)
plot_faces("6 Reconstructed faces using inverse_transform", X_hat, 3, 2)

```

</details>


### Exercise 4

Use the **similar_distance** as input for the **threshold** function. You can specify the min and max threshold for the distances. Return the index of the original dataset and the query dataset that satisfy the specified threshold. Use **plot_data_query** to plot the first 5 pairs of face images that our image retrieval system "views" as similar.


In [ ]:
o_index, q_index = threshold(similar_distance)

Length=64
Width=64
plot_data_query(o_index, q_index, 5)

<details>
    <summary>Click here for Solution</summary>

```python
X_hat_M = w6@H
plot_faces("6 Reconstructed faces using matrix operations", X_hat_M, 3,2)
print("-------------------------------------------------------")
plot_faces("6 Reconstructed faces using inverse_transform", X_hat, 3, 2)
```

</details>
